In [4]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


saveSteps = False
lowThreshold = 0
numberOfIterations = 5
image = cv2.imread('img.png')

In [5]:
def update():
	img_copy = image.copy()
	gray = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)
	_, thresh = cv2.threshold(gray, lowThreshold, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
	kernel = np.ones((3, 3), np.uint8)
	background = cv2.dilate(thresh, kernel, iterations=numberOfIterations)
	afterDistanceTransformation = cv2.distanceTransform(background, 1, 5)
	afterDistanceTransformation = np.uint8(afterDistanceTransformation)
	_, foreground = cv2.threshold(afterDistanceTransformation, lowThreshold, 255, 0)

	foreground = np.uint8(foreground)
	unknownRegions = cv2.subtract(background, foreground)

	_, markers = cv2.connectedComponents(foreground)
	markers = markers + 1

	markers[unknownRegions == 255] = 0

	markers = cv2.watershed(img_copy, markers)

	img_copy[markers == -1] = (0, 0, 255)
	cv2.imshow("img", img_copy)

	if saveSteps:
		plt.rcParams['figure.dpi'] = 250
		plt.subplot(241)
		plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)),
		plt.title('Original')
		plt.axis('off')
		plt.subplot(242)
		plt.imshow(thresh, cmap='gray'),
		plt.title('Threshold')
		plt.axis('off')
		plt.subplot(243)
		plt.imshow(background, cmap='gray'),
		plt.title('Dilate')
		plt.axis('off')
		plt.subplot(244)
		plt.imshow(afterDistanceTransformation, cmap='gray'),
		plt.title('Dist Transform')
		plt.axis('off')
		plt.subplot(245)
		plt.imshow(foreground, cmap='gray'),
		plt.title('Threshold')
		plt.axis('off')
		plt.subplot(246)
		plt.imshow(unknownRegions, cmap='gray'),
		plt.title('Unknow')
		plt.axis('off')
		plt.subplot(247)
		plt.imshow(np.abs(markers), cmap='jet'),
		plt.title('Markers')
		plt.axis('off')
		plt.subplot(248)
		plt.imshow(cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB)),
		plt.title('Result')
		plt.axis('off')
		plt.savefig('steps.png')


def changeLowThresholdValue(val):
	global lowThreshold
	lowThreshold = val
	update()


def changeNumberOfIterations(val):
	global numberOfIterations
	numberOfIterations = val
	update()

In [6]:
cv2.namedWindow('Control')
cv2.createTrackbar('thresh low', 'Control', 20, 255, changeLowThresholdValue)
cv2.createTrackbar('iterations', 'Control', 5, 100, changeNumberOfIterations)
cv2.waitKey(0)
cv2.destroyAllWindows()